# This notebook shows the implementation of document QnA system in Langchain

---



In [1]:
!pip install -qU "langchain[google-genai]" langchain-google-genai langchain-chroma pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.5 MB/s eta 0:00:

Using Gemini as the LLM model:

In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


Enter API key for Google Gemini: ··········


# Pipeline step 1: Upload

In [3]:
from google.colab import files  # Only works in Colab

uploaded_files = files.upload()  # You'll be prompted to upload 3 PDFs


Saving jesc110.pdf to jesc110.pdf
Saving jesc112.pdf to jesc112.pdf
Saving jesc111.pdf to jesc111.pdf


In [5]:
!pip install -U langchain-community
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00


# Pipeline step 2: Chunking

In [6]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

all_docs = []

for filename in uploaded_files.keys():
    loader = PyPDFLoader(filename)
    pdf_docs = loader.load()
    all_docs.extend(pdf_docs)

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(all_docs)


# Pipeline step 3: Generate embeddings and vector storage

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = Chroma(
    collection_name="new_example_collection",
    embedding_function=embeddings,
    persist_directory="./new_chroma_langchain_db"
)

vector_store.add_documents(chunks)


['bcd575f9-7891-42a4-806d-bd5ad2b5c4d1',
 'a1c16b86-4cf6-4b12-84d0-473fee5a8e49',
 '6cdf55ac-9103-4c09-ba79-1cffd0b416f3',
 'a552ba89-1618-4487-bfce-8352eb682be5',
 'bbd27faa-4f58-4057-91d8-361b8d32e4ce',
 '5153c201-6a08-4bbe-9f83-9756efafd35e',
 '1c41201a-6420-48e6-be0a-76735dea63da',
 '668f3ee5-2f72-4a00-9d0f-371db290b1e8',
 '2fccfbec-743e-47b1-8992-ae609d66d78b',
 '0861febe-ce8f-49f0-956e-8ad06ffeb979',
 '3697100d-a1ab-418d-b2db-364dae834c88',
 '1c055a56-7a64-41b9-a291-60bffb6b0a32',
 '959008a8-2f88-4a89-b517-73d5f71626a6',
 '803a00f6-a19b-4d42-9260-dc8c57e8f7fd',
 '37acce5a-2c52-4817-a11f-f07b31d9862d',
 '97118380-3415-4841-8bac-a00f33f0648f',
 'a8f0f12c-8b0f-40e1-9e7d-dac2c4f88a6a',
 '2a69b04a-fcc3-47a6-8409-8d46ba54439b',
 '4ebde557-4192-41bd-85b8-eeb88ddf6ae8',
 '4db7a910-8804-4b4a-9f2c-bdc34bbd3c83',
 '416252bc-cdc9-4aa3-99ea-5e5ae1978bfb',
 '9480f6db-0f9f-407e-b8e0-1999b6b0a2c9',
 '570bf1fd-a281-44ff-89ce-0a578b7719ca',
 '43f23cbc-cf4c-4724-af8d-1cb3fde13914',
 '1250fced-771b-

# Pipeline step 4: Retrieval and answer generation

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.chains import RetrievalQA

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

retriever = vector_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

query = "Summarize the main idea of the uploaded PDFs."
result = qa_chain.run(query)

print("Q:", query)
print("A:", result)

Q: Summarize the main idea of the uploaded PDFs.
A: Based on the provided snippets, the uploaded PDFs seem to be related to:

*   **Electrical circuits and concepts:** Including current, resistance, parallel and series connections.
*   **Applications of electricity:** Such as electric lamps, heating devices, and appliances (TV, toaster, electric heater).
*   **Energy consumption calculations.**
*   **Material properties:** Specifically, the use of tungsten and alloys in electrical applications.
*   **Experimental observations:** Activity involving ammeter readings and material components.


In [10]:
query = "what is light? how can humans see?"
result = qa_chain.run(query)

print("Q:", query)
print("A:", result)

Q: what is light? how can humans see?
A: Light is something that humans use to see objects around them. The human eye has a lens in its structure that enables sight.


In [11]:
query = "write a short note on electricty. who discovered it?"
result = qa_chain.run(query)

print("Q:", query)
print("A:", result)

Q: write a short note on electricty. who discovered it?
A: Electricity has an important place in modern society as a controllable and convenient form of energy. Oersted showed that electricity and magnetism were related phenomena when he observed that an electric current passed through a metallic wire placed nearby. His research later created technologies such as the radio, television and fiber optics.


In [18]:
query = "tell about different eye defects in detail"
result = qa_chain.run(query)

print("Q:", query)
print("A:", result)

Q: tell about different eye defects in detail
A: There are mainly three common refractive defects of vision:

1.  **Myopia:** Also known as near-sightedness. A person with myopia can see nearby objects clearly but cannot see distant objects distinctly.
2.  **Hypermetropia:** Also known as far-sightedness.
3.  **Presbyopia:**


In [19]:
query = "how is magnetism and electricity related?"
result = qa_chain.run(query)

print("Q:", query)
print("A:", result)

Q: how is magnetism and electricity related?
A: Electricity and magnetism are linked to each other. Electric current through a copper wire produces a magnetic effect. Also, there is the possibility of an electric effect of moving magnets.


In [21]:
query = "what is light made up of? what are its componenets?"
result = qa_chain.run(query)

print("Q:", query)
print("A:", result)

Q: what is light made up of? what are its componenets?
A: Sunlight is made up of seven colours. The splitting of light into its component colours is called dispersion. When white light is dispersed, it produces a spectrum.
